In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**CÀI ĐẶT VÀ IMPORT THƯ VIỆN**

In [ ]:
%cd '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/'
!ls

/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR
CrossValidationScore.py		  Predictions.py
cv_split.pickle			  predict.py
data				  __pycache__
EDA.ipynb			  README.md
initial_test_b3_fold3_clahe	  split_folds.ipynb
logs				  submit
logs_final			  submit_final
MNIST				  submit_initial_test_b3.csv
model_concate_10_no_epoch.txt	  submit_oanh_initial_test_b3.csv
model_concate_15epoch_weight.txt  submit_vessel_initial_test_b3.csv
model_concate_19epoch.txt	  train_1cycle.py
model_concate_20epoch.txt	  train_all_folds_args.sh
model_concate_5_no_epoch.txt	  train_all_folds.sh
model_concate_6epoch.txt	  train.py
model_concate_epoch_20.txt	  utils.py
odir_submit.py			  Visualization.ipynb


In [ ]:
!pip install catalyst==20.2.4
!pip install tqdm==4.33
!pip install efficientnet_pytorch
!pip install pytorch_toolbelt

In [ ]:
from sklearn import svm
from keras.callbacks import CSVLogger
from keras.callbacks import LambdaCallback
from utils import *
import pandas as pd
import sys
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from torch.utils.data import Dataset, DataLoader
import tensorflow as tf
import torch
import collections
from pytorch_toolbelt.inference import tta
from catalyst.dl.callbacks import InferCallback
from catalyst.dl.runner import SupervisedRunner
from torch.nn.functional import softmax
from catalyst.dl.callbacks import EarlyStoppingCallback, AccuracyCallback, F1ScoreCallback, ConfusionMatrixCallback, MixupCallback
from catalyst.contrib.nn.schedulers import OneCycleLR, ReduceLROnPlateau, StepLR, MultiStepLR
from torchvision import transforms
import torch.nn.functional as F
from torch.autograd import Variable
from torchsummary import summary
import pickle
import time
import copy

Using TensorFlow backend.
alchemy not available, to install alchemy, run `pip install alchemy-catalyst`.


In [ ]:
splits = pickle.load(open('/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/cv_split.pickle', 'rb'))
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/splited_train_new.csv')
labels  = ['N','D','G','C','A','H','M','O']
n_classes = len(labels)
fold_idx, batch_size, model_name, image_size, head_n_epochs, head_lr, full_n_epochs, full_lr, exp_name = 3, 32, 'efficientnet-b3', 240, 5, 1e-5, 25, 3e-4, 'initial_test_b3'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
num_classes = len(labels)
seed_everything(1234)
runner = SupervisedRunner()
model = prepare_model(model_name, n_classes)

Will load efficientnet-b3 with 8 classes


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth



Loaded pretrained weights for efficientnet-b3


**TRAIN ẢNH VESSEL**

In [ ]:
train_path = '/content/gdrive/My Drive/Colab Notebooks/retina/Retina-VesselNet/experiments/VesselNet/test/vessel-train-full/'
valid_path = '/content/gdrive/My Drive/Colab Notebooks/retina/Retina-VesselNet/experiments/VesselNet/test/vessel-train-full/'
train_dataset = EyeDataset(dataset_path = train_path, 
					labels=data.loc[splits['train_idx'][fold_idx],labels].values, 
					ids=data.loc[splits['train_idx'][fold_idx],'id'].values, 
					albumentations_tr=aug_train_heavy(image_size)) 
val_dataset = EyeDataset(dataset_path=valid_path, 
					labels=data.loc[splits['test_idx'][fold_idx],labels].values, 
					ids=data.loc[splits['test_idx'][fold_idx],'id'].values, 
					albumentations_tr=aug_val(image_size))
train_loader =  DataLoader(train_dataset,
					num_workers=8,
					pin_memory=False,
					batch_size=batch_size,
					shuffle=True)
val_loader =  DataLoader(val_dataset,
					num_workers=8,
					pin_memory=False,
					batch_size=batch_size,
					shuffle=True)	
loaders = collections.OrderedDict()
loaders["train"] = train_loader
loaders["valid"] = val_loader
logdir = 'logs_final/{}_fold{}_vessel/'.format(exp_name, fold_idx)
print('Training only head for {} epochs with inital lr {}'.format(head_n_epochs, head_lr))
for p in model.parameters():
		p.requires_grad = False
for p in model._fc.parameters():
		p.requires_grad = True
optimizer = torch.optim.Adam(model.parameters(), lr=head_lr)
criterion = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer=optimizer, factor=0.75, patience=2)
runner.train(model=model,
			criterion=criterion,
			optimizer=optimizer,
			loaders=loaders,
			logdir=logdir,
			scheduler=scheduler,
			callbacks=[
                MixupCallback()
				],
			num_epochs=head_n_epochs,
			verbose=True)      
print('Train whole net for {} epochs with initial lr {}'.format(full_n_epochs, full_lr))
print('Train whole net for {} epochs with initial lr {}'.format(full_n_epochs, full_lr))
for p in model.parameters():
		p.requires_grad = True
optimizer = torch.optim.Adam(model.parameters(), lr=full_lr)
criterion = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer=optimizer, factor=0.5, patience=5)
runner.train(model=model,
			criterion=criterion,
			optimizer=optimizer,
			loaders=loaders,
			logdir=logdir,
			scheduler=scheduler,
			callbacks=[
                MixupCallback()
				],
			num_epochs=full_n_epochs,
			verbose=True)

Training only head for 5 epochs with inital lr 1e-05
1/5 * Epoch (train): 100% 164/164 [06:09<00:00,  2.25s/it, loss=2.060]
1/5 * Epoch (valid): 100% 55/55 [01:57<00:00,  1.29it/s, loss=2.032]
[2020-08-02 13:04:56,985] 
1/5 * Epoch 1 (train): _base/lr=1.000e-05 | _base/momentum=0.9000 | _timers/_fps=325.9118 | _timers/batch_time=1.9915 | _timers/data_time=1.9324 | _timers/model_time=0.0580 | loss=2.0341
1/5 * Epoch 1 (valid): _base/lr=1.000e-05 | _base/momentum=0.9000 | _timers/_fps=569.5941 | _timers/batch_time=1.8669 | _timers/data_time=1.8302 | _timers/model_time=0.0366 | loss=2.0148
2/5 * Epoch (train): 100% 164/164 [02:19<00:00,  1.18it/s, loss=1.830]
2/5 * Epoch (valid): 100% 55/55 [00:33<00:00,  3.28it/s, loss=1.971]
[2020-08-02 13:07:51,246] 
2/5 * Epoch 2 (train): _base/lr=1.000e-05 | _base/momentum=0.9000 | _timers/_fps=140.2361 | _timers/batch_time=0.7086 | _timers/data_time=0.5005 | _timers/model_time=0.2049 | loss=1.9202
2/5 * Epoch 2 (valid): _base/lr=1.000e-05 | _base/mo

**TEST VESSEL**

In [ ]:
test_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/XYZ_ODIR.csv')
test_data_left = test_data.copy()
test_data_right = test_data.copy()
test_data_left.loc[:,'id'] = test_data_left.ID.apply(lambda x: str(x)+'_left.jpg')
test_data_right.loc[:,'id'] = test_data_left.ID.apply(lambda x: str(x)+'_right.jpg')
test_data = pd.concat([test_data_left,test_data_right])
test_data.sort_values(['ID'],inplace=True)
test_path = '/content/gdrive/My Drive/Colab Notebooks/retina/Retina-VesselNet/experiments/VesselNet/test/vessel-test-full/'
test_dataset = EyeDataset(dataset_path=test_path, 
                         labels=test_data.loc[:,labels].values, 
                         ids=test_data.loc[:,'id'].values, 
                         albumentations_tr=aug_val(image_size))
test_loader =  DataLoader(test_dataset,
                         num_workers=8,
                         pin_memory=False,
                         batch_size=batch_size,
                         shuffle=False)  
loaders = collections.OrderedDict()
loaders["valid"] = test_loader    
probabilities_list = []
ttatype='d4'
logdir = 'logs_final/initial_test_b3_fold3_vessel/'
model = prepare_model(model_name, n_classes)
model.cuda()
model.load_state_dict(torch.load(os.path.join(logdir, '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs_final/initial_test_b3_fold3_vessel/checkpoints/best.pth'))['model_state_dict'])
model.eval()
if ttatype == 'd4':
    model = tta.TTAWrapper(model, tta.d4_image2label)
elif ttatype == 'fliplr_image2label':
    model = tta.TTAWrapper(model, tta.d4_image2label)
runner = SupervisedRunner(model=model)
runner.infer(model=model,loaders=loaders,callbacks=[InferCallback()])
predictions = runner.callbacks[0].predictions['logits']
probabilities = softmax(torch.from_numpy(predictions),dim=1).numpy()    	
for idx in range(probabilities.shape[0]):	
    if all(probabilities[idx,:]<0.5):	
        probabilities[idx,0] = 1.0	
probabilities_list.append(probabilities)	
probabilities_combined = np.stack(probabilities_list,axis=0).mean(axis=0) 	
predicted_labels = pd.DataFrame(probabilities_combined, columns=labels)	
predicted_labels['id'] = test_data.loc[:,'id'].values	
predicted_labels.loc[:,'ID'] = predicted_labels.id.apply(lambda x: x.split('_')[0])	
predicted_labels_groupped = predicted_labels.groupby(['ID']).aggregate(dict(zip(labels,['max']*(len(labels)))))	
print(type(predicted_labels_groupped))
predicted_labels_groupped['ID'] = predicted_labels_groupped.index.values.astype(int)	
predicted_labels_groupped.reset_index(drop=True, inplace=True)	
predicted_labels_groupped.sort_values('ID',inplace=True)	
predicted_labels_groupped = predicted_labels_groupped.loc[:,['ID']+labels] 	
predicted_labels_groupped.to_csv('/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/submit_final/initial_test_b3_fold3_vessel.csv', index=False)

Will load efficientnet-b3 with 8 classes
Loaded pretrained weights for efficientnet-b3
<class 'pandas.core.frame.DataFrame'>
